## Chinese Loans to African Countries by Natural Resource Reserves

## Setup

In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Import Data

In [179]:
chinese_dev_finance = pd.read_excel(r'AidData_GCDFD_v2.xlsx', sheet_name='AidData_GCDFD_v2')
rare_earths = pd.read_csv(r'./ree/main.csv')

In [176]:
# group by country
rare_earth_country = rare_earths.groupby(['country']).size().reset_index()

# Filter out certain flow types
chinese_dev_finance_grants = chinese_dev_finance[chinese_dev_finance['Flow Type'] == 'Grant']
chinese_dev_finance_loans = chinese_dev_finance[chinese_dev_finance['Flow Type'] == 'Loan']

# TO-DO : Reconcile country names (some are different in both data sets)

In [181]:
# Clean duplicate + regional names
names_to_be_replaced = {'Viet Nam': 'Vietnam', 'Congo (Zaire)': 'Congo', 'Surinam': 'Suriname', 'Burma': 'Myanmar', 'Kyrgyz Republic': 'Kyrgyzstan', "Democratic People's Republic of Korea": 'North Korea'}
for old_name, new_name in names_to_be_replaced.items():
    chinese_dev_finance['Recipient'] = chinese_dev_finance['Recipient'].replace([old_name],new_name)
    rare_earth_country['country'] = rare_earth_country['country'].replace([old_name],new_name)

names_to_delete = ['Asia, regional', 'Oceania, regional', 'Europe, regional', 'America, regional', 'Middle East, regional']
for name in names_to_delete:
    chinese_dev_finance = chinese_dev_finance[chinese_dev_finance['Recipient'] != name]

x = chinese_dev_finance['Recipient'].tolist()
y = rare_earth_country['country'].tolist()
print(list(set(x) - set(y))) # in China finance dataset but not in rare earths dataset
print(list(set(y) - set(x))) # in rare earths dataset but not in China dataset

['Pakistan', 'Eritrea', 'Belarus', 'Niue', 'Cabo Verde', 'Panama', 'Micronesia', 'Yemen', 'Brunei Darussalam', 'Maldives', 'Niger', 'Chile', 'Comoros', 'West Bank and Gaza Strip', 'Tonga', 'Moldova', 'Jamaica', 'Guinea-Bissau', 'Africa, regional', 'French Polynesia', 'Jordan', 'Seychelles', 'Guam', 'Iraq', 'Sao Tome and Principe', 'Israel', 'Tunisia', 'Democratic Republic of the Congo', 'El Salvador', 'Bahamas', 'North Macedonia', 'Oman', 'Azerbaijan', 'Togo', 'Gambia', 'Haiti', 'Curacao', 'Lebanon', 'Vanuatu', 'Georgia', 'Cuba', 'Marshall Islands', 'Palau', 'South Sudan', 'Costa Rica', 'Bosnia and Herzegovina', 'Fiji', 'Turkmenistan', 'Nepal', 'Equatorial Guinea', 'Timor-Leste', 'Montenegro', 'Peru', 'Grenada', 'Papua New Guinea', 'Mauritius', 'Barbados', 'Trinidad and Tobago', 'Saint Lucia', 'Ethiopia', 'Kiribati', 'Albania', 'Sudan', 'Cambodia', 'Algeria', 'Nauru', 'Antigua and Barbuda', 'Central African Republic', 'Lesotho', "Lao People's Democratic Republic", 'Djibouti', 'Iran', '

# Run OLS

In [182]:
def combine_dataframes (cfd_df, rare_earth_df, sep_year=False, cfd_dropna=True, rare_earth_dropna=False): 
    cfd_df = cfd_df[['Recipient', 'Commitment Year', 'Amount (Constant USD2017)']]
    if not sep_year:
        cfd_df = cfd_df.groupby(['Recipient'])['Amount (Constant USD2017)'].sum().reset_index()
    else:
        cfd_df = cfd_df.groupby(['Recipient', 'Commitment Year'])['Amount (Constant USD2017)'].sum().reset_index()
    rare_earth_df = rare_earth_df.groupby(['country']).size().reset_index()

    # rename columns
    cfd_df = cfd_df.rename(columns={'Recipient':'country'})
    rare_earth_df = rare_earth_df.rename(columns={0: 'Rare Earth Deposits'})

    # deal with null values
    if cfd_dropna and rare_earth_dropna:
        result = pd.merge(cfd_df, rare_earth_df, on='country')
    elif cfd_dropna:
        result = pd.merge(cfd_df, rare_earth_df, on='country', how='left')
    elif rare_earth_dropna:
        result = pd.merge(cfd_df, rare_earth_df, on='country', how='right')
    else:
        result = pd.merge(cfd_df, rare_earth_df, on='country', how='outer')
    result = result.fillna(0)

    return result

In [183]:
def run_ols (df, x_vars='Rare Earth Deposits', y_vars='Amount (Constant USD2017)', constant=True):
    x = df[x_vars].tolist()
    y = df[y_vars].tolist()

    if constant:
        x = sm.add_constant(x)

    # performing the regression
    # and fitting the model
    result = sm.OLS(y, x).fit()
  
    # printing the summary table
    print(result.summary())

In [184]:
# all recipients, all flows
combined_agg = combine_dataframes(chinese_dev_finance, rare_earths)
run_ols(combined_agg)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.264
Method:                 Least Squares   F-statistic:                     53.33
Date:                Tue, 01 Feb 2022   Prob (F-statistic):           1.72e-11
Time:                        21:23:55   Log-Likelihood:                -3714.7
No. Observations:                 147   AIC:                             7433.
Df Residuals:                     145   BIC:                             7439.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.493e+09   2.03e+09      3.192      0.0

In [185]:
# all countries w/ rare earths, all flows
combined_agg_allcountries = combine_dataframes(chinese_dev_finance, rare_earths, cfd_dropna=False)
run_ols(combined_agg_allcountries)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     3.848
Date:                Tue, 01 Feb 2022   Prob (F-statistic):             0.0515
Time:                        21:24:03   Log-Likelihood:                -4235.7
No. Observations:                 167   AIC:                             8475.
Df Residuals:                     165   BIC:                             8482.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.192e+09   2.05e+09      4.475      0.0

In [186]:
# all recipients, only loans
combined_onlyloans = combine_dataframes(chinese_dev_finance_loans, rare_earths)
run_ols(combined_onlyloans)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.275
Model:                            OLS   Adj. R-squared:                  0.270
Method:                 Least Squares   F-statistic:                     51.27
Date:                Tue, 01 Feb 2022   Prob (F-statistic):           4.68e-11
Time:                        21:24:12   Log-Likelihood:                -3450.5
No. Observations:                 137   AIC:                             6905.
Df Residuals:                     135   BIC:                             6911.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        4.85e+09   1.94e+09      2.506      0.0

In [187]:
# all recipients, only grants
combined_onlygrants = combine_dataframes(chinese_dev_finance_grants, rare_earths)
run_ols(combined_onlygrants)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.3014
Date:                Tue, 01 Feb 2022   Prob (F-statistic):              0.584
Time:                        21:24:20   Log-Likelihood:                -3120.3
No. Observations:                 142   AIC:                             6245.
Df Residuals:                     140   BIC:                             6251.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.384e+08   7.64e+07      4.432      0.0

In [188]:
# all recipients w/rare earths
combined_recip_with_re = combine_dataframes(chinese_dev_finance, rare_earths, rare_earth_dropna=True)
run_ols(combined_recip_with_re)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.233
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     16.41
Date:                Tue, 01 Feb 2022   Prob (F-statistic):           0.000165
Time:                        21:24:22   Log-Likelihood:                -1436.6
No. Observations:                  56   AIC:                             2877.
Df Residuals:                      54   BIC:                             2881.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        8.66e+09   5.63e+09      1.539      0.1